In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
data = pd.read_csv('../openreview.csv')
decision_to_binary = {'Accept (Poster)': 1, 'Accept (Oral)': 1, 'Accept (Talk)': 1, 'Accept (Spotlight)': 1, 'Invite to Workshop Track': 0, 'Withdrawn': 0, 'Reject': 0}

In [3]:
def logistic_regression():
    scores=[]
    google_indicator=[]
    microsoft_indicator=[]
    facebook_indicator=[]
    target=[]
    count = 0
    for ratings, decision, institution, year in zip(data['ratings'], data['decisions'], data['institution'], data['year']):
        if pd.notnull(institution) and pd.notnull(ratings) and year == 2021:

            rates = ratings.split(';')
            rates=[int(x) for x in rates]
            rating_avg = np.average(rates)

            institutions = institution.split(';')
            institution_set = set(institutions)

            binary_decision=decision_to_binary.get(decision)

            for inst in institution_set:
                scores.append(rating_avg)
                target.append(binary_decision)

                if 'Google' in inst or 'Deepmind' in inst or 'Deep Mind' in inst:
                    google_indicator.append(1)
                    facebook_indicator.append(0)
                    microsoft_indicator.append(0)
                elif 'Facebook' in inst:
                    google_indicator.append(0)
                    facebook_indicator.append(1)
                    microsoft_indicator.append(0)
                elif 'Microsoft' in inst:
                    google_indicator.append(0)
                    facebook_indicator.append(0)
                    microsoft_indicator.append(1)
                else:
                    google_indicator.append(0)
                    facebook_indicator.append(0)
                    microsoft_indicator.append(0)
    
    x = pd.DataFrame()
    #x['mean reviewer score'] = scores
    x['google'] = google_indicator
    x['microsoft'] = microsoft_indicator
    x['facebook'] = facebook_indicator
    x['constant'] = [1]*len(scores)

    print(np.sum(google_indicator), np.sum(facebook_indicator), np.sum(microsoft_indicator))
    y=np.array(target)

    logit_model = sm.Logit(y, x)
    summary = logit_model.fit().summary()

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=0)
    logisticRegr = LogisticRegression(random_state=0)
    model = logisticRegr.fit(x_train, y_train)
    accuracy = logisticRegr.score(x_test, y_test)
    
    print('Count: ',count)
    return summary, accuracy

In [4]:
summary, score = logistic_regression()
print(summary)
print('Accuracy: ', score)

354 151 158
Optimization terminated successfully.
         Current function value: 0.611902
         Iterations 5
Count:  0
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 8116
Model:                          Logit   Df Residuals:                     8112
Method:                           MLE   Df Model:                            3
Date:                Sat, 02 Oct 2021   Pseudo R-squ.:                0.005042
Time:                        11:13:53   Log-Likelihood:                -4966.2
converged:                       True   LL-Null:                       -4991.4
Covariance Type:            nonrobust   LLR p-value:                 6.775e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
google         0.7050      0.110      6.428      0.000       0.490       0.920
microso